In [131]:
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cacki\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

def major_anaysis(text):

    essay_input_corpus = str(text) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)

    skip_gram = 1
    workers = multiprocessing.cpu_count()
    bigram_transformer = Phrases(split_sentences)

    model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

    model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=300)
    
    tech_eng = ['General Engineering', 'Mechanical Engineering', 'Industrial Engineering', 'Bioengineering',
                'Operations Research', 'Materials Science', 'Electrical Engineering', 'Computer Science/Engineering',
                'Civil Engineering', 'Aerospace Engineering', 'Chemical Engineering', 'Environmental Engineering']

    math_sci = ['Agriculture', 'Physics', 'Biology', 'Biophysics', 'Biochemistry/Molecular Biology',
                'Mathematics', 'Applied Mathematics', 'Nutrition/Food Science', 'Genetics', 'Astronomy/Astrophysics',
                'Statistics', 'Chemistry', 'Neuroscience', 'Economics','Nursing','Pre-Med', 'Pre-Veterinary'
                'Pharmacy/Pre-Pharmacy', 'Pre-Dental', 'Pharmacy', 'Pre-Pharmacy']

    social_sci = ['Economics Asian Studies', 'African Studies','Psychology', "Women's Studies",'Anthropology', 'Political Science/Government',
                    'Political Science', 'Political Government', 'International Relations/Affairs', 'International Relations',
                    'International Affairs', 'Public Policy/Affairs', 'Public Policy', 'Public Affairs', 'Public Health',
                    'Environmental Science/Studies', 'Environmental Science', 'Environmental Studies', 'Urban Planning', 'Education']

    Humanities = ['Archeology', 'Film Studies','German (Language and Literature)','Art History',
                    'French (Language and Literature)','Spanish','Theology/Religious Studies',
                    'Linguistics','History','English Language and Literature','Comparative Literature',
                    'Philosophy','Classical Studies/Latin/Greek']

    visualart = ['Film&Television', 'Film', 'Television', 'Design (Graphic, Industrial, Computer Graphics)','Graphic Design','Industral Design', 'Computer Graphics', 'Fashion Design', 'Photography', 'Architecture']

    business = ['Business', 'Finance', 'Accounting', 'Marketing', 'Intl', 'Biz','International Business','Mgmt.', 'Info Systems', 'Hotel/Hospitality Management',
                'Hotel Management', 'Hospitality Management']

    music = ['Music (Performance)', 'Music', 'Music Theory']

    performing_arts = ['performing arts']
    
    major_filter_tech_eng = tech_eng
    
    ####문장에 setting_words_filter_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
    #우선 토큰화한다.
    retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
    token_input_text = retokenize.tokenize(essay_input_corpus)
    #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
    #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
    filtered_setting_text = []
    for k in token_input_text:
        for j in major_filter_tech_eng :
            if k == j:
                filtered_setting_text.append(j)
    
    #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
    
    filtered_setting_text_ = set(filtered_setting_text) #중복제거
    filtered_setting_text__ = list(filtered_setting_text_) #다시 리스트로 변환
    print (filtered_setting_text__) # 중복값 제거 확인
    
    for i in filtered_setting_text__:
        ext_setting_sim_words_key = model.most_similar_cosmul(i) #모델적용
    
    setting_total_count = len(filtered_setting_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 setting 표현 수
    setting_count_ = len(filtered_setting_text__) #중복제거된 setting표현 총 수
        
    result_setting_words_ratio = round(setting_total_count/total_words * 100, 2)
    return result_setting_words_ratio, total_sentences, total_words, setting_total_count, setting_count_


In [132]:
# 3개의 입력: 전공 3개
majors = """ Asian Studies,Film Studies, Archeology  """
major_list_ = majors.split(",")
#소문자로 변환
major_list = []
for mjl in major_list_:
    mjl_ = mjl.lower()
    mjl__ = mjl_.strip()
    major_list.append(mjl__)

major_list



['asian studies', 'film studies', 'archeology']

In [133]:
# 아래와 같이 입력해야 함, 소문자 처리, 공백처리 완료!
#    Chemical Engineering, investment,biomedical  
#awards = """ winner of ABC competition competition, none """
# 2개의 입력: 수상내역, 성적   carnegie international student essay contest	    winner of ABC competition competition
awards = """  carnegie international student essay contest, Extremely Selective	 """
honor_list_ = awards.split(",")
#입력 문자를 소문자로 변환
honor_list = []
for hor in honor_list_:
    hor_ = hor.lower()
    hor__ = hor_.strip()
    honor_list.append(hor__)

honor_list

['carnegie international student essay contest', 'extremely selective']

In [134]:
#입력된 수상결과(수상관련 문장)가 어떤 리스트에 속하는지 비교(사용자의 선택전공입력 - 사용자의 수상결과 - 시스템에서 관련 전공제안)

In [135]:
import pandas as pd

#Awards 데이터 불러오기
data_awards = pd.read_csv('awards_list.csv')
data_awards['award_title'] = data_awards['award_title'].str.lower() #소문자로 변환해야 타이틀을 비교할 수 있음(매칭되는지)
data_awards['Big Major Category #1'] = data_awards['Big Major Category #1'].str.lower()
data_awards['Big Major Category #2'] = data_awards['Big Major Category #2'].str.lower()
data_awards['Big Major Category #3'] = data_awards['Big Major Category #3'].str.lower()
data_awards['Big Major Category #4'] = data_awards['Big Major Category #4'].str.lower()
data_awards

,award_title,award_words,major_keywords,vision_contents,prestige,prestige_score,Big Major Category #1,Big Major Category #2,Big Major Category #3,Big Major Category #4
0,all-american high school film festival,NaN,No Extra Match,Y,Extremely Selective,5.0,communication,visualart,NaN,NaN
1,annual architectural design competition for hi...,Architectural Foundation of San Francisco High...,No Extra Match,Y,Very Selective,4.0,visualart,NaN,NaN,NaN
2,atlantic and college board writing prize,NaN,No Extra Match,Y,Very Selective,4.0,humanities,NaN,NaN,NaN
3,ayn rand institute contest,NaN,No Extra Match,Y,Very Selective,4.0,humanities,NaN,NaN,NaN
4,bennington young writers award,NaN,No Extra Match,Y,Very Selective,4.0,humanities,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
883,make for change,NaN,No Extra Match,Y,Extremely Selective,5.0,tech/engineering,business,social science,NaN
884,volunteerthon,NaN,No Extra Match,Y,Extremely Selective,5.0,social science,NaN,NaN,NaN
885,uci-gati science journal,NaN,No Extra Match,Y,Very Selective,4.0,math/science,tech/engineering,NaN,NaN
886,wico,World Invention Creativity Olympiad,No Extra Match,Y,Selective,3.0,tech/engineering,NaN,NaN,NaN


In [136]:
is_honor_list = data_awards['award_title'] == honor_list[0] # 입력한 결과중 수상내역 문자열 과 데이터프레임의 타이클 컬럼에서 같은 조건
re_honor = data_awards[is_honor_list] # 데이터 필터링하여 새로운 변수에 저장
re_honor # 결과 출력(여기서 결과가 나오면, 수상내역 출력, 없으면... )

,award_title,award_words,major_keywords,vision_contents,prestige,prestige_score,Big Major Category #1,Big Major Category #2,Big Major Category #3,Big Major Category #4
7,carnegie international student essay contest,Carnegie Council,No Extra Match,Y,Extremely Selective,5.0,social_sci,humanities,NaN,NaN


In [137]:
award_anaysis_re = re_honor[["award_title","prestige_score","Big Major Category #1","Big Major Category #2","Big Major Category #3","Big Major Category #4"]]
award_anaysis_re


,award_title,prestige_score,Big Major Category #1,Big Major Category #2,Big Major Category #3,Big Major Category #4
7,carnegie international student essay contest,5.0,social_sci,humanities,NaN,NaN


In [138]:
# major fit을 계산하려면 입력한 희망전공 3개를 예: ['mechanical engineering', ' computer science', ' bioligy']

# 각 전공을 다음 리스트에서 검색하여 매칭되는 값이 일치(여기서는 business이면) 하면 'FIT',  아니라면 ''NOT FIT 없으면 '?'로 표시


tech_eng_ = ['mechanical engineering', 'general Engineering', 'mechanical Engineering', 'Industrial engineering', 'bioengineering',
            'operations Research', 'materials Science', 'electrical Engineering', 'computer Science/engineering','computer engineering'
            'civil engineering', 'aerospace Engineering', 'chemical Engineering', 'environmental engineering', 'stem', 'steam', 'robotics', 
            'robots', 'computer', 'computers', 'computer programming', 'programming', 'coding', 'computing', 'mechanical', 'mechanics', 
            'mechanism', 'ai', 'artificial intelligence', 'sensor', 'environmental engineering', 'environmental technology', 
            'alternative energy', 'renewable energy', 'material science', 'biomedical', 'biomaterials', 'nanotech', 'pharmaceutical',
            'biomechanics', 'biomimicking', 'biomimicry', 'machine', 'bioengineering', 'chemical engineering', 'applied physics', 
            'operations research', 'civil engineering', 'aerospace', 'system', 'operations research', 'industrial engineering', 'computer Science'
            'invention', 'electrical', 'computational']

tech_eng = []
for i in tech_eng_:
    i_ = i.lower()
    tech_eng.append(i_)

math_sci_ = ['Agriculture', 'Physics', 'Biology', 'Biophysics', 'Biochemistry/Molecular Biology',
            'Mathematics', 'Applied Mathematics', 'Nutrition/Food Science', 'Genetics', 'Astronomy/Astrophysics',
            'Statistics', 'Chemistry', 'Neuroscience', 'Economics','Nursing','Pre-Med', 'Pre-Veterinary'
            'Pharmacy/Pre-Pharmacy', 'Pre-Dental', 'Pharmacy', 'Pre-Pharmacy', 'STEM', 'STEAM', 'Chemistry', 'physics',
            'biology', 'bio', 'biochem', 'biochemistry', 'science', 'math', 'mathematics', 'algorithm', 'biophysics', 
            'applied math', 'genetics', 'scientific', 'astronomy', 'astrophysics', 'statistics', 'neuroscience', 'dental',
            'doctor', 'physical', 'medical', 'scientific research', 'Biomaterials', 'material science', 'nutrition', 'clinical',
            'earth science', 'environmental science', 'ecology', 'ecological', 'veterinary', 'molecular', 'nanotech', 'biomedical',
            'pharmaceutical', 'food science, biotech', 'brain', 'botany', 'botanical', 'immunology', 'immune', 'zoology', 'biomechanics',
            'biomimicking', 'biomimicry']

math_sci = []
for j in math_sci_:
    j_ = j.lower()
    math_sci.append(j_)
    
social_sci = ['Economics Asian Studies', 'African Studies','Psychology', "Women's Studies",'Anthropology', 'Political Science/Government',
                'Political Science', 'Political Government', 'International Relations/Affairs', 'International Relations',
                'International Affairs', 'Public Policy/Affairs', 'Public Policy', 'Public Affairs', 'Public Health',
                'Environmental Science/Studies', 'Environmental Science', 'Environmental Studies', 'Urban Planning', 'Education',
                'Volunteer', 'community', 'community service', 'culture', 'equality', 'service',' social issue', 'social', 
                'society', 'economic', 'economy', 'SDGs', 'environment', 'policy', 'psychology', 'Asian studies', 'race relations', 
                'ethnicity', 'ethnic', 'women', 'anthropology', 'politic', 'international relations', 'international affairs',
                'diplomacy', 'diplomat', 'public policy', 'public health,' 'inequality', 'geography', 'geographic', 'linguistic', 
                 'sociology', 'culture', 'urban studies', 'diversity', 'LGBT', 'LGBTQ', 'sustainable', 'sustainability', 'debate',
                 'Middle East',' Middle Eastern Studies']

social_sci = []
for k in social_sci_:
    k_ = k.lower()
    social_sci.append(k_)

humanities_ = ['Archeology', 'Film Studies','German (Language and Literature)','Art History',
                'French (Language and Literature)','Spanish','Theology/Religious Studies',
                'Linguistics','History','English Language and Literature','Comparative Literature',
                'Philosophy','Classical Studies/Latin/Greek', 'English', 'literature', 'history', 'writing', 
              'heritage', 'essay', 'script', 'culture' ,'humanities', 'poem', 'poet', 'playwright', 'script',
              'poetry', 'literary', 'lit', 'writing', 'Spanish', 'German', 'Chinese', 'Italian', 'French', 'Russian', 'literature', 
              'Latin', 'Greek', 'anthropology', 'archaeology', 'Law', 'ethics', 'film studies']

humanities = []
for l in humanities_:
    l_ = l.lower()
    humanities.append(l_)
    

visualart_ = ['Film&Television', 'Film', 'Television', 'Design (Graphic, Industrial, Computer Graphics)','Graphic Design','Industral Design', 
             'Computer Graphics', 'Fashion Design', 'Photography', 'Architecture', 'Movie', 'visual', 'music', 'art', 'arts', 'performing', 
             'paint', 'painting',' visual arts', 'design', 'draw', 'drawing', 'graphic', 'architecture', 'architect', 'film', 'sculpture',
             'artwork', 'fine art']

visualart = []
for m in visualart_:
    m_ = m.lower()
    visualart.append(m_)
    

business_ = ['Business', 'Finance', 'Accounting', 'Marketing', 'Intl', 'Biz','International Business','Mgmt.', 'Info Systems', 
            'Hotel/Hospitality Management', 'Hotel Management', 'Hospitality Management', 'Business', 'start up', 'start-up',
            'entrepreneur', 'entrepreneurship', 'investment', 'invest', 'business plan', 'finance', 'marketing',' organizational behavior',
            'logistics', 'management', 'managing', 'social entrepreneur', 'social enterprise', 'accounting', 'managerial', 'MIS', 
            'Management Information Systems', 'Hotel management', 'hospitality']

business = []
for o in business_:
    o_ = o.lower()
    business.append(o_)
    

music_ = ['Music (Performance)', 'Music', 'Music Theory']

music = []
for p in music_:
    p_ = p.lower()
    music.append(p_)
    

performing_arts_ = ['performing arts','performance', 'dance', 'Movie', 'music', 'musical', 'theater', 'theatre', 'opera', 'thespian', 'song',
                   'sing', 'vocal', 'drama', 'orchestra', 'band', 'acting', 'acrobatics', 'ballet','circus', 'magic', 'mime', 'puppetry',
                   'ventriloquism', 'spoken word', 'stand-up comedy', 'Chinese opera', 'pansori', 'chamber music', 'cabaret', 
                   'jazz band', 'big band', 'Bharatanatyam', 'Gamelan semar pegulingan', 'Shōmyō', 'Odissi', 'Cantonese opera']

performing_arts = []
for q in performing_arts_:
    q_ = q.lower()
    performing_arts.append(q_)

communication_ = ['Documentary', 'SNS', 'Social Network', 'PR', 'public relations', 'advertisement', 'advertising', 'movie', 'film', 'news',
                 'journalism', 'newspaper', 'debate', 'speech', 'newspaper', 'media', 'communication']

communication = []
for r in communication_:
    r_ = r.lower()
    communication.append(r_)

In [139]:
major_list

['asian studies', 'film studies', 'archeology']

In [140]:
df_a = pd.DataFrame(tech_eng, columns=["major"])
df_a["main_major"] = "tech_eng"
df_a[:5]

,major,main_major
0,mechanical engineering,tech_eng
1,general engineering,tech_eng
2,mechanical engineering,tech_eng
3,industrial engineering,tech_eng
4,bioengineering,tech_eng


In [141]:
df_b = pd.DataFrame(math_sci, columns=["major"])
df_b["main_major"] = "math_sci"
df_b[:5]

,major,main_major
0,agriculture,math_sci
1,physics,math_sci
2,biology,math_sci
3,biophysics,math_sci
4,biochemistry/molecular biology,math_sci


In [142]:
df_c = pd.DataFrame(social_sci, columns=["major"])
df_c["main_major"] = "social_sci"
df_c[:5]

,major,main_major
0,economics asian studies,social_sci
1,african studies,social_sci
2,psychology,social_sci
3,women's studies,social_sci
4,anthropology,social_sci


In [143]:
df_d = pd.DataFrame(humanities, columns=["major"])
df_d["main_major"] = "humanities"
df_d[:5]

,major,main_major
0,archeology,humanities
1,film studies,humanities
2,german (language and literature),humanities
3,art history,humanities
4,french (language and literature),humanities


In [144]:
df_e = pd.DataFrame(business, columns=["major"])
df_e["main_major"] = "business"
df_e[:5]

,major,main_major
0,business,business
1,finance,business
2,accounting,business
3,marketing,business
4,intl,business


In [145]:
df_f = pd.DataFrame(music, columns=["major"])
df_f["main_major"] = "music"
df_f[:5]

,major,main_major
0,music (performance),music
1,music,music
2,music theory,music


In [146]:
df_g = pd.DataFrame(performing_arts, columns=["major"])
df_g["main_major"] = "performing_arts"
df_g[:5]

,major,main_major
0,performing arts,performing_arts
1,performance,performing_arts
2,dance,performing_arts
3,movie,performing_arts
4,music,performing_arts


In [147]:
df_h = pd.DataFrame(communication, columns=["major"])
df_h["main_major"] = "communication"
df_h[:5]

,major,main_major
0,documentary,communication
1,sns,communication
2,social network,communication
3,pr,communication
4,public relations,communication


In [148]:
# 전공별 데이터와 주 전공을 하나의 데이터프레임으로 합친다. 
total_major_df = pd.concat([df_a,df_b,df_c,df_d,df_e,df_f,df_g,df_h])
total_major_df

,major,main_major
0,mechanical engineering,tech_eng
1,general engineering,tech_eng
2,mechanical engineering,tech_eng
3,industrial engineering,tech_eng
4,bioengineering,tech_eng
...,...,...
12,debate,communication
13,speech,communication
14,newspaper,communication
15,media,communication


In [149]:
#이제 main_major에서 관련 전공을 선택하고, 어떤 분야의 main_major인지 매칭해서 fit, not fit, ? 를 구분한다.
major_list # 다시 확인. 그리고나서 이것을 total_major_df에서 검색을 한다.  매칭되면 main_major와 fit한지 판단. 

['asian studies', 'film studies', 'archeology']

In [150]:
honor_list # 다시 확인  이것은 수상한 내용임.

['carnegie international student essay contest', 'extremely selective']

In [151]:
award_anaysis_re  #다시확인.. 이것도 추출한 내용임 여기서, 주목해야 할 것은 Big Major Category #1 의 Business 임. 
#이 상은 Business와 관련성이 있다는 의미임
#이 결과를 total_major_df에서 major 에서 원하는 전공을 main_major에서 찾아본 후, 
# main_major에 business 가 있다면 fit, 아니면 not fit, 없다면 ? 으로 처리

,award_title,prestige_score,Big Major Category #1,Big Major Category #2,Big Major Category #3,Big Major Category #4
7,carnegie international student essay contest,5.0,social_sci,humanities,NaN,NaN


In [152]:
# 우선 Big Major Category #1 를 확인해보자. 
award_anaysis_re["Big Major Category #1"]

7    social_sci
Name: Big Major Category #1, dtype: object

In [153]:
#추출한 결과를 리스트에 저장
major_ctgs_ =[]
major_ctg = list(award_anaysis_re["Big Major Category #1"])
major_ctgs_.append(major_ctg)
major_ctg = list(award_anaysis_re["Big Major Category #2"])
major_ctgs_.append(major_ctg)
major_ctg = list(award_anaysis_re["Big Major Category #3"])
major_ctgs_.append(major_ctg)
major_ctg = list(award_anaysis_re["Big Major Category #4"])
major_ctgs_.append(major_ctg)

In [154]:
 major_ctgs = [y for x in major_ctgs_ for y in x]

In [155]:
 major_ctgs

['social_sci', 'humanities', nan, nan]

In [156]:
#total_major_df[total_major_df['major'] == 'agriculture'] # 테스트.. 잘 찾아지는구만

In [157]:
# 원하는 전공 리스트의 값을 데이터프레임 total_major_df에서 찾아보자
fit_analysis_list = []
for search_major in major_list:
    
    re__ = total_major_df[total_major_df['major'] == search_major]  # 데이터프레임에 찾고자하는 전공이 있다면
    print (re__) # 결과물을 리스트에 저장해보자
    fit_analysis_list.append(re__)



            major  main_major
35  asian studies  social_sci
           major  main_major
1   film studies  humanities
43  film studies  humanities
        major  main_major
0  archeology  humanities


In [158]:
fit_analysis_list

[            major  main_major
 35  asian studies  social_sci,
            major  main_major
 1   film studies  humanities
 43  film studies  humanities,
         major  main_major
 0  archeology  humanities]

In [159]:
#fit_analysis_list 를 문자로 만들어서 개별 리스트로 만들자
s =" ".join(map(str, fit_analysis_list))
s

'            major  main_major\n35  asian studies  social_sci            major  main_major\n1   film studies  humanities\n43  film studies  humanities         major  main_major\n0  archeology  humanities'

In [160]:
type(s) #타입확인

str

In [174]:
fit_list_str = s.split()               # str을 공란()으로 쪼갬   
fit_list_str

['major',
 'main_major',
 '35',
 'asian',
 'studies',
 'social_sci',
 'major',
 'main_major',
 '1',
 'film',
 'studies',
 'humanities',
 '43',
 'film',
 'studies',
 'humanities',
 'major',
 'main_major',
 '0',
 'archeology',
 'humanities']

In [176]:
#드디어 분석 fit analysis!!!!
#선택한 전공 3개, 수상내역 1개를 비교하여 데이터베이스 비교 분석하여 상과 전공이 일치하는지 판단!
fit_result = []
for category in fit_list_str:
    for major_item in major_ctgs:
        if major_item  == category:
            print ("fit")
            fit_result.append('fit')
        else:
            print ("not sure")
            fit_result.append('not sure')

not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
fit
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
fit
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
fit
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
not sure
fit
not sure
not sure


In [177]:
fit_result #리스트로 변환
df_fit_re = pd.DataFrame(fit_result)
df_fit_re

,0
0,not sure
1,not sure
2,not sure
3,not sure
4,not sure
...,...
79,not sure
80,not sure
81,fit
82,not sure


In [178]:
df_fit_re.drop_duplicates() #중복값 제거!!!! 최종 결과 도출, 여기서 FIT > NOT FIT  or NOT SURE 구분하면 됨

,0
0,not sure
20,fit


In [165]:
########### Award Analysis  #############

In [166]:
#수상내역 데이터 테이블을 불러온다아아앙
data_awards[:10]

,award_title,award_words,major_keywords,vision_contents,prestige,prestige_score,Big Major Category #1,Big Major Category #2,Big Major Category #3,Big Major Category #4
0,all-american high school film festival,NaN,No Extra Match,Y,Extremely Selective,5.0,communication,visualart,NaN,NaN
1,annual architectural design competition for hi...,Architectural Foundation of San Francisco High...,No Extra Match,Y,Very Selective,4.0,visualart,NaN,NaN,NaN
2,atlantic and college board writing prize,NaN,No Extra Match,Y,Very Selective,4.0,humanities,NaN,NaN,NaN
3,ayn rand institute contest,NaN,No Extra Match,Y,Very Selective,4.0,humanities,NaN,NaN,NaN
4,bennington young writers award,NaN,No Extra Match,Y,Very Selective,4.0,humanities,NaN,NaN,NaN
5,blue ocean entrepreneurship competition,NaN,No Extra Match,Y,Very Selective,4.0,business,NaN,NaN,NaN
6,caribou mathematics competition,NaN,No Extra Match,Y,Selective,3.0,math_sci,NaN,NaN,NaN
7,carnegie international student essay contest,Carnegie Council,No Extra Match,Y,Extremely Selective,5.0,social_sci,humanities,NaN,NaN
8,cet and nshss scholarship for gap programs,NaN,No Extra Match,Y,Selective,3.0,social_sci,NaN,NaN,NaN
9,chapman art and writing holocaust contest,Annual Holocaust Art and Writing Contest,"art, film",Y,Very Selective,4.0,visualart,NaN,NaN,NaN


In [167]:
# 수상 포지션 용어를 추출 분석하기 >>> 일단 이 수상내역을

honor_list

['carnegie international student essay contest', 'extremely selective']

In [168]:
#수상리스트의 값을 점수로 계산해보자  extremely selective 같은 경우는 그냥 데이프레임에서 추출하면 됨
#단, winner of ABC competition competition, none  입력을 받았을 경우에는 데이터베이스에 수상내역이 없기 때문에 예외처리를 해야 한다.
is_honor_list = data_awards['award_title'] == honor_list[0] # 입력한 결과중 수상내역 문자열 과 데이터프레임의 타이클 컬럼에서 같은 조건
re_honor = data_awards[is_honor_list] # 데이터 필터링하여 새로운 변수에 저장
re_honor # 결과 출력(여기서 결과가 나오면출력, 만약 수상내역 출력, 없으면... 예외처리를 해야함.. 2번째 방식으로 분석해야 함)

,award_title,award_words,major_keywords,vision_contents,prestige,prestige_score,Big Major Category #1,Big Major Category #2,Big Major Category #3,Big Major Category #4
7,carnegie international student essay contest,Carnegie Council,No Extra Match,Y,Extremely Selective,5.0,social_sci,humanities,NaN,NaN


In [169]:
#점수를 추출한다. 그리고 이 점수는 입력한 수상내역의 문자를 확인하여 '예외의 룰'을 적용!!!
award_anaysis_re = re_honor[["prestige_score"]]
award_anaysis_re

,prestige_score
7,5.0


In [170]:
award_anaysis_re = award_anaysis_re.isnull().any().any() #부울 값을 반환한다. 값이 없으니까  Fasle이다.
award_anaysis_re #찾는 값이 없기 때문에  2차 분석을 시작!

False

In [171]:
type(award_anaysis_re)

numpy.bool_

In [172]:
def second_aw_score_anaysis(input_award_title_second):
    remove_null_text = input_award_title_second.strip() #앞 뒤 공백 제거한다. 입력할 때 공백을 넣을 수 있기 때문에.. 에러방지
    split_awards_list = remove_null_text.split(" ")
    #결과를 토대로 점수 계산하기위해서 data_awards_score의 데이터 프레임에서 해당 점수값 도출하기
    #그러기 위해서 우선 입력한 단어(수상설명)을 토대로
    
    # 타이틀에 명칭을 분석해서 점수 계산하기
    # 우선 타이틀 별 점수로직 테이블 불러오기

    # Extremely Selective : 5
    # Very Selective : 4
    # Selective : 3
    # Mediocre : 2
    # Low : 1

    # *** Winner / Won 등의 단어가 위의 수상 포지션 없는 상태에서 쓰였을 경우 그 상에 우승 했다는 뜻이므로 강도 100% 적용
    # 예를 들면 winner of ABC competition 이나 Won ABC competition 이러면 1등이죠… 다만, Won 2nd Place 또는 2nd place winner 이러면 2등인거니깐여


    data_awards_score = pd.read_csv('award_title_score.csv')
    data_awards_score ['title'] = data_awards_score ['title'].str.lower() #소문자로 변환해야 타이틀을 비교할 수 있음(매칭되는지)
    data_awards_score.set_index('title', inplace=True) #title을 인뎃스로 변환, 그래야 값을 찾기 쉽다.
    
    #입력한 수상내역의 리스트 값을 하나씩 불러와서 데이터프레이에 있는지 비교 찾아내서 해당 점수를 가져오기
    get_score__ = []
    for award_word in split_awards_list: #데이터프레임에서 인덱스의 값과 비교하여
        if award_word in data_awards_score.index: #df에 특정 단어가 있다면, 해당하는 컬럼의 값을 가져오기
            get_score_ = data_awards_score.loc[award_word,'fin_score']
            get_score__.append(get_score_)
            #print ("got it")

        else:
            #print ("can not calculate")
            pass
        
    ##### 입력한 수상내역에 해당하는 최종 점수 계산
    return print (get_score__) #해당 점수는 다음과 같음

In [173]:
if award_anaysis_re == False: # 값이 없으면 False 이기 땜문에 2차 분석 시작해야 함
    print ("Score is :")
    second_aw_score_anaysis(honor_list[0])

Score is :
[]


## 끝!!!

해결해야 할 사항 >> 수상명칭 입력할 때 수상명칭외에 다른 단어를 넣을 때를 계산하려면, 수상명칭을 정확히 일치한 것을 계산하거나
입력 내용을 모두 단어로 분리해서 가장 단어가 적합하게(일치한 횟수를 백분율로 계산) 나온 것을 계산(높은 값)에 해당하면 특정 대상의 수상결과가 되는 것으로 해야, 수상내역외에 추가 단어를 동시에 분석이 가능하다.

## 수상명 2차 분석


## 우선 단어로 분리해서계산하고, 위에서 분석하는 수상명 전체를 이용해서 분석하는 기능은 나중에 합칠 것!!!!!!!!!!!!!!!!!!!

In [216]:
#수상명에 1st prize 라는 단어가 들어가있음, 공백문자포함
input_award_title_second = """ winner of ABC competition competition """ 

In [217]:
remove_null_text = input_award_title_second.strip() #앞 뒤 공백 제거한다. 입력할 때 공백을 넣을 수 있기 때문에.. 에러방지
split_awards_list = remove_null_text.split(" ")
split_awards_list #결과를 토대로 점수 계산하기위해서 data_awards_score의 데이터 프레임에서 해당 점수값 도출하기
#그러기 위해서 우선 입력한 단어(수상설명)을 토대로 

['winner', 'of', 'ABC', 'competition', 'competition']

In [218]:
# 타이틀에 명칭을 분석해서 점수 계산하기
# 우선 타이틀 별 점수로직 테이블 불러오기

# Extremely Selective : 5
# Very Selective : 4
# Selective : 3
# Mediocre : 2
# Low : 1

# *** Winner / Won 등의 단어가 위의 수상 포지션 없는 상태에서 쓰였을 경우 그 상에 우승 했다는 뜻이므로 강도 100% 적용
# 예를 들면 winner of ABC competition 이나 Won ABC competition 이러면 1등이죠… 다만, Won 2nd Place 또는 2nd place winner 이러면 2등인거니깐여


data_awards_score = pd.read_csv('award_title_score.csv')
data_awards_score ['title'] = data_awards_score ['title'].str.lower() #소문자로 변환해야 타이틀을 비교할 수 있음(매칭되는지)
data_awards_score.set_index('title', inplace=True) #title을 인뎃스로 변환, 그래야 값을 찾기 쉽다.
data_awards_score

,score_cal_rate,fin_score
title,,
extremely selective,5,5.0
very selective,4,4.0
selective,3,3.0
mediocre,2,2.0
low,1,1.0
first,100,5.0
first place,100,5.0
first prize,100,5.0
top,100,5.0


#입력한 수생내역의 리스트 값을 하나씩 불러와서 데이터프레이에 있는지 비교 찾아내서 해당 점수를 가져오기
get_score__ = []
for award_word in split_awards_list:
    get_score = data_awards_score ['title'] == award_word
    get_score__.append(get_score_)
    
    

In [219]:
split_awards_list

['winner', 'of', 'ABC', 'competition', 'competition']

In [220]:
#입력한 수생내역의 리스트 값을 하나씩 불러와서 데이터프레이에 있는지 비교 찾아내서 해당 점수를 가져오기
get_score__ = []
for award_word in split_awards_list: #데이터프레임에서 인덱스의 값과 비교하여
    if award_word in data_awards_score.index: #df에 특정 단어가 있다면, 해당하는 컬럼의 값을 가져오기
        get_score_ = data_awards_score.loc[award_word,'fin_score']
        get_score__.append(get_score_)
        print ("got it")
        
    else:
        print ("can not calculate")
        


got it
can not calculate
can not calculate
can not calculate
can not calculate


In [221]:
##### 입력한 수상내역에 해당하는 최종 점수 계산
get_score__ #해당 점수는 다음과 같음 

[5.0]

In [222]:
#수상리스트의 값을 점수로 계산해보자  extremely selective 같은 경우는 그냥 데이프레임에서 추출하면 됨
is_honor_list = data_awards['award_title'] == honor_list[0] # 입력한 결과중 수상내역 문자열 과 데이터프레임의 타이클 컬럼에서 같은 조건
re_honor = data_awards[is_honor_list] # 데이터 필터링하여 새로운 변수에 저장
re_honor # 결과 출력(여기서 결과가 나오면, 수상내역 출력, 없으면... )

,award_title,award_words,major_keywords,vision_contents,prestige,prestige_score,Big Major Category #1,Big Major Category #2,Big Major Category #3,Big Major Category #4
7,carnegie international student essay contest,Carnegie Council,No Extra Match,Y,Extremely Selective,5.0,social science,humanities,NaN,NaN


#이렇게 하는 방법도 있지만.... 이건 바보같은 코드 ㅎㅎ
#################################################

# major_list를 하나씩 추출해서 리스트로 만든 필터와 비교한다.
try:
    for major_item in major_list:
        for tech_eng_item in tech_eng:
            if major_item == tech_eng_item:
                print("FIT tech_eng")
                raise NotImplementedError
            else:
                for math_sci_item in math_sci:
                    if major_item == math_sci_item:
                        print('FIT math_sci')
                        raise NotImplementedError
                    else:
                        for social_sci_item in social_sci:
                            if major_item == social_sci_item:
                                print ("FIT social_sci")
                                raise NotImplementedError
                            else:
                                for humanities_item in humanities:
                                    if major_item == humanities_item:
                                        print("FIT humanities")
                                        raise NotImplementedError
                                    else:
                                        for visualart_item in visualart:
                                            if major_item == visualart_item:
                                                print("FIT visualart")
                                                raise NotImplementedError
                                            else:
                                                for business_item in business:
                                                    if major_item == business_item:
                                                        print("FIT  business")
                                                        raise NotImplementedError
                                                    else:
                                                        for music_item in music:
                                                            if major_item == music_item:
                                                                print("FIT music")
                                                                raise NotImplementedError
                                                            else:
                                                                for performing_arts_item in performing_arts:
                                                                    if major_item == performing_arts_item:
                                                                        print("FIT performing")
                                                                        raise NotImplementedError
                                                                    else:
                                                                        for communication_item in communication:
                                                                            if major_item == communication_item:
                                                                                print("FIT communication")
                                                                                raise NotImplementedError
                                                                            else:
                                                                                print("NOT FIT!!!!!")
                                                                                raise NotImplementedError
                                                                            
except:
    print("end")

In [223]:
major_anaysis(input_text)

NameError: name 'major_anaysis' is not defined

In [ ]:
data_major_list = pd.read_csv('major_list.csv')
data_major_list[:5]

In [ ]:
#Summer Activity 불러오기
data_summer_activity = pd.read_csv('summer_activity.csv')
data_summer_activity[:5]